In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
# #         print(os.path.join(dirname, filename

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# !pip install focal-loss

# Ideas
1. Perform a binary model for healthy vs not healthy. Then check for the illness if it's not healthy.
    1. Don't run it on Complex. 

In [ ]:
import pandas as pd
import tensorflow as tf
import sklearn

import numpy as np
import os
import cv2
import PIL
import sklearn
import matplotlib.pyplot
from sklearn.model_selection import StratifiedShuffleSplit
import time
import collections
from collections import Counter
import matplotlib.pyplot as plt
# from augmentations import *
# from helper import *
import tensorflow_addons as tfa
import albumentations
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Cutout)
import csv

In [ ]:
main_dir = '../input/resized-plant2021/img_sz_640/'
imgs_dir = os.path.join(main_dir)
labels_path = os.path.join('../input/plant-pathology-2021-fgvc8/', 'train.csv')

df = pd.read_csv(labels_path)

image_names = df['image']
image_names = [os.path.join(imgs_dir, fname) for fname in image_names]
df['image'] = image_names

unique_labels = sorted(df['labels'].unique())


# print("Total classes : ", num_classes)
print("Total Samples : ", len(df))

In [ ]:
# labels that will be used for validation
final_labels_for_submission = ['complex', 'frog_eye_leaf_spot', 'frog_eye_leaf_spot complex', 'healthy', 'powdery_mildew', 'powdery_mildew complex', 'rust', 'rust complex', 'rust frog_eye_leaf_spot', 'scab', 'scab frog_eye_leaf_spot', 'scab frog_eye_leaf_spot complex',
                                'complex frog_eye_leaf_spot', 'complex powdery_mildew', 'complex rust', 'frog_eye_leaf_spot rust', 'frog_eye_leaf_spot scab', 'scab complex frog_eye_leaf_spot', 'frog_eye_leaf_spot scab complex', 'frog_eye_leaf_spot complex scab', 'complex frog_eye_leaf_spot scab', 'complex scab frog_eye_leaf_spot']

In [ ]:
unique_labels
single_labels = []
for label in unique_labels:
    if ' ' not in label:
        single_labels.append(label)

unique_labels = single_labels.copy()
num_classes = len(unique_labels)


In [ ]:
# Loading 2020 dataset
path_2020 = '../input/plant-pathology-2020-fgvc7'
df_2020 = pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')

columns = df_2020.columns.tolist()
columns[2] = 'complex'
columns[0] = 'image'

df_2020.columns = columns

image_filenames_2020 = df_2020['image'].tolist()
image_filenames_2020 = [os.path.join(path_2020, 'images', name) + '.jpg' for name in image_filenames_2020]

df_2020['image'] = image_filenames_2020

classes_2020 = ['healthy', 'complex', 'rust', 'scab']
classes_to_add = []
for lbl in unique_labels:
    if lbl not in classes_2020:
        classes_to_add.append(lbl)

classes_2020 = sorted(classes_2020 + classes_to_add)

df_2020[classes_to_add[0]] = [0] * len(df_2020)
df_2020[classes_to_add[1]] = [0] * len(df_2020)

a = df_2020.loc[3, unique_labels].tolist()
labels_cat_2020 = []
for i in range(len(df_2020)):
    current_row = df_2020.loc[i, unique_labels].tolist()
    current_label = []
    for i, row in enumerate(current_row):
        if row:
            current_label.append(unique_labels[i])
    labels_cat_2020.append(''.join(current_label))    
df_2020['underlying_labels'] = labels_cat_2020
df_2020['labels'] = df_2020['underlying_labels'].apply(lambda x: x.split())

In [ ]:
df_2020.head()

In [ ]:
class_labels = df['labels'].tolist().copy()
df['labels'] = df['labels'].apply(lambda x : x.split(' '))
df['underlying_labels'] = class_labels

# Remove duplicates

In [ ]:
import pickle
dict_file = open("../input/plant-pathology-2021-duplicates/duplicates.pickle",'rb')
duplicates = pickle.load(dict_file)
duplicates_to_remove = []
for key,val in duplicates.copy().items():
    if not val:
        del duplicates[key]
    else:
        duplicates_to_remove.extend(val)
duplicates_to_remove = set(duplicates_to_remove)

duplicates_mask = df['image'].apply(lambda x: x.split('/')[-1] not in duplicates_to_remove)
print('Length of DF before removing duplicates : {}'.format(len(df)))
df = df[duplicates_mask]
print('Length of DF after removing duplicates : {}'.format(len(df)))


# Create Multi label for each class

In [ ]:
for i, class_name in enumerate(unique_labels):
    df[class_name] = df['labels'].apply(lambda x : int(class_name in x))

# Model evaluation metrics
1. Correlation matrix divergence
2. Adversarial Validations importances

# For ensembling
1. Boosting using LightGBM


# Hyper parameters


In [ ]:
img_size = 512
num_folds = 5
seed = 42

IMG_SIZE = img_size
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE= 32
tfdata_buffer_size = 1000
learning_rate = 0.001

In [ ]:
use_2020_dataset = True
if use_2020_dataset:
    df = pd.concat([df, df_2020], axis=0)

In [ ]:

df['fold'] = 0

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
mskf = StratifiedShuffleSplit(n_splits=num_folds, random_state=seed, test_size=0.2)
for i, (train_index, test_index) in enumerate(mskf.split(df['image'], df['labels'])):
    df.iloc[test_index, -1] = i
df['fold'] = df['fold'].astype('int')
    
mask = df['fold'] == 4
df['is_valid'] = mask


train_df = df[df['is_valid'] == False]
val_df = df[df['is_valid'] == True]

class_mapping = {}

for i in range(len(unique_labels)):
    class_mapping[unique_labels[i]] = i

classes = list(class_mapping.keys())

label_to_cat_label_map = {}
for key, value in class_mapping.items():
    label_to_cat_label_map[value] = key

# train_df['encoded_labels'] = train_df['underlying_labels'].apply(lambda x: class_mapping[x])
# val_df['encoded_labels'] = val_df['underlying_labels'].apply(lambda x : class_mapping[x])



In [ ]:
train_filenames = train_df['image'].tolist()
# train_filenames = [os.path.join(imgs_dir, fname) for fname in train_filenames]
# train_labels = tf.keras.utils.to_categorical(train_df['encoded_labels'].tolist())
train_labels = train_df[unique_labels].to_numpy()
val_filenames = val_df['image'].tolist()
# val_filenames = [os.path.join(imgs_dir, fname) for fname in val_filenames]
# val_labels = tf.keras.utils.to_categorical(val_df['encoded_labels'].tolist())
val_labels = val_df[unique_labels].to_numpy()

train_cat_labels = train_df['underlying_labels'].tolist()
val_cat_labels = val_df['underlying_labels'].tolist()


test_filenames = os.listdir('../input/plant-pathology-2021-fgvc8/test_images/')
test_filenames = [os.path.join('../input/plant-pathology-2021-fgvc8/test_images/', name) for name in test_filenames]

In [ ]:
print('Train samples : {}'.format(len(train_filenames)))
print('Validation samples : {}'.format(len(val_filenames)))

In [ ]:
def view_image(ds):
    try:
        image, cat_label, filename, class_name = next(iter(ds)) # extract 1 batch from the dataset
    except ValueError as e:
        image, cat_label = next(iter(ds))
    image = image.numpy()
    cat_label = cat_label.numpy()

    fig = plt.figure(figsize=(22, 22))
    for i in range(8):
        ax = fig.add_subplot(4, 2, i+1, xticks=[], yticks=[])
        ax.imshow(image[i])
        ax.set_title(f"Label: {cat_label[i]}")

# Image Preprocessing

In [ ]:
def load_image(img):
    return img

###############################################################################
def init_grabcut_mask(h, w):
    mask = np.ones((h, w), np.uint8) * cv2.GC_PR_BGD
    mask[h//4:3*h//4, w//4:3*w//4] = cv2.GC_PR_FGD
    mask[2*h//5:3*h//5, 2*w//5:3*w//5] = cv2.GC_FGD
    return mask

def remove_background(image):
    h, w = image.shape[:2]
    mask = init_grabcut_mask(h, w)
    bgm = np.zeros((1, 65), np.float64)
    fgm = np.zeros((1, 65), np.float64)
    cv2.grabCut(image, mask, None, bgm, fgm, 1, cv2.GC_INIT_WITH_MASK)
    mask_binary = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    result = cv2.bitwise_and(image, image, mask = mask_binary)
#     add_contours(result, mask_binary) # optional, adds visualizations
    return result

def tf_remove_background(image):
    image_without_background = tf.numpy_function(remove_background, inp=[image], Tout=[image.dtype])
    return image_without_background[0]

##################################################################

def image_resize(image, img_size, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = (img_size, img_size)
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

def remove_local_average_color(img):
    # print("Removing local average color")
    scale = 300
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # OpenCV processes in BGR
    output_bgr = cv2.addWeighted(img_bgr, 4, cv2.GaussianBlur(img_bgr, (0, 0), scale / 10), -4, 128)
    output_rgb = cv2.cvtColor(output_bgr, cv2.COLOR_BGR2RGB)
    return output_rgb

def clahe_LAB(img):
    # print("CLAHE LAB")
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # OpenCV processes in BGR
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    output_bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    output_rgb = cv2.cvtColor(output_bgr, cv2.COLOR_BGR2RGB)
    return output_rgb

def clahe_HSV(img):
    # print("CLAHE HSV")
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # OpenCV processes in BGR
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    hsv_planes = cv2.split(hsv)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    hsv_planes[2] = clahe.apply(hsv_planes[2])
    hsv = cv2.merge(hsv_planes)
    output_bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    output_rgb = cv2.cvtColor(output_bgr, cv2.COLOR_BGR2RGB)
    return output_rgb


# Create a base train and val dataloader

In [ ]:
def get_image_dontnormalize(filename, categorical_label, image_size, class_name, preprocess_method=None):
    # decode_jpeg
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string)  # decodes to uint8 tensor
#     image_no_background = tf_remove_background(image_decoded)
    image_resized = tf.cast(tf.image.resize(image_decoded, (image_size, image_size)),
                            tf.uint8)  # model expects int [0,255]

    return image_resized, categorical_label, filename, class_name

def create_tf_dataset(image_size,
                      filenames,
                      categ_labels,
                      class_name,
                      normalize_fn=None,
                      preprocess_type='none'):
   
    image_size_tiled = tf.constant([image_size] * len(filenames))
    preprocess_type_tiled = tf.constant([preprocess_type] * len(filenames))
    ds = tf.data.Dataset.from_tensor_slices(
        (tf.constant(filenames), tf.constant(categ_labels), image_size_tiled,
         tf.constant(class_name),
         preprocess_type_tiled)) \
        .map(normalize_fn, num_parallel_calls=AUTOTUNE)
    return ds

def create_tf_map_dataset(image_size, filenames, categ_labels, class_name, normalize_fn, preprocess_type='none'):
    '''Uses the given filenames and labels to create tf datasets.'''
    train_ds = create_tf_dataset(image_size, filenames, categ_labels, class_name, normalize_fn)
    return train_ds

train_ds = create_tf_map_dataset(IMG_SIZE, train_filenames, train_labels, train_cat_labels, normalize_fn=get_image_dontnormalize,)
val_ds = create_tf_map_dataset(IMG_SIZE, val_filenames, val_labels, val_cat_labels, normalize_fn=get_image_dontnormalize, )

################################################################
def create_sampled_dataset(train_ds, unique_labels):
    df_s = []
    for value in unique_labels:
        temp_ds = train_ds.filter(lambda image, categ_label, filename, class_name : class_name == value)
        df_s.append(temp_ds)
    return df_s
    
desired_fraction = np.array([1/ len(class_mapping)] * len(class_mapping))
dfs = create_sampled_dataset(train_ds, unique_labels)

use_balanced_data = False
if use_balanced_data:
    train_data = tf.data.experimental.sample_from_datasets(dfs, desired_fraction)
    steps_per_epoch = max(1, 2 * len(train_filenames) // BATCH_SIZE)
else:
    steps_per_epoch = max(1, len(train_filenames) // BATCH_SIZE)

################################################################



# View Class based images

In [ ]:
# class_names = 'scab frog_eye_leaf_spot complex'
# temp_df = dfs[class_mapping[class_names]].batch(32)
# view_image(temp_df)

In [ ]:
'''
Use mixup on 

1. Rust + Complex
2. scab + frog_eye_leaf_spot
3. frog_eye_leaf_spot + complex
4. rust + frog_eye_leaf_spot
5. powdert_mildew + complex
6. scab frog_eye_leaf_spot complex
'''

# Batching


In [ ]:
train_data = train_ds.shuffle(buffer_size=tfdata_buffer_size, reshuffle_each_iteration=True,
                             seed=0)
train_data = train_data.batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_data = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

train_data = train_data.map(lambda images, categ_labels, filename, class_name: (images, categ_labels))
val_data_with_filenames = val_data.map(lambda images, categ_labels, filename, class_name: (images, class_name, filename))
val_data = val_data.map(lambda images, categ_labels, filename, class_name: (images, categ_labels))


In [ ]:
TOTAL_PERCENT_OF_DATA = 100
def data_split(images, labels, percent_batch_to_augment):
    # split the batch with random indices
    bs = tf.shape(images)[0]
    indices = tf.range(bs)
    shuf_indices = indices
    # shuf_indices = tf.random.shuffle(indices, seed)
    # shuf_indices = tf.range(bs)

    if percent_batch_to_augment == 0:
        raise NotImplementedError

    elif percent_batch_to_augment <= 50 or percent_batch_to_augment == 100:
        unaltered_data = tf.gather(images,
                                   shuf_indices[:(bs - (bs // (TOTAL_PERCENT_OF_DATA // percent_batch_to_augment)))])
        aug_data = tf.gather(images, shuf_indices[(bs - (bs // (TOTAL_PERCENT_OF_DATA // percent_batch_to_augment))):])

        unaltered_label = tf.gather(labels,
                                    shuf_indices[:(bs - (bs // (TOTAL_PERCENT_OF_DATA // percent_batch_to_augment)))])
        aug_label = tf.gather(labels, shuf_indices[(bs - (bs // (TOTAL_PERCENT_OF_DATA // percent_batch_to_augment))):])

    elif 50 < percent_batch_to_augment < 100:
        percent_batch_to_augment = 100 - percent_batch_to_augment
        aug_data = tf.gather(images, shuf_indices[:(bs - (bs // (TOTAL_PERCENT_OF_DATA // percent_batch_to_augment)))])
        unaltered_data = tf.gather(images,
                                   shuf_indices[(bs - (bs // (TOTAL_PERCENT_OF_DATA // percent_batch_to_augment))):])

        aug_label = tf.gather(labels, shuf_indices[:(bs - (bs // (TOTAL_PERCENT_OF_DATA // percent_batch_to_augment)))])
        unaltered_label = tf.gather(labels,
                                    shuf_indices[(bs - (bs // (TOTAL_PERCENT_OF_DATA // percent_batch_to_augment))):])

    else:
        raise NotImplementedError

    return unaltered_data, aug_data, unaltered_label, aug_label
def img_flip(image, label, percent_batch_to_augment=100):
    """Flip augmentation
    Args:
        percent_batch_to_augment:
        image: image to flip [bs, img_size, img_size, 3]
        label: image label (not used)

    Returns:
        Flipped image, label (shuffled as per augmented image batch)
    """
    print("Using Flip Aug")

    # split the data into unaltered data and data to be augmented
    original_data, aug_data, original_label, aug_label = data_split(image, label, percent_batch_to_augment)

    # keep original_data untouched
    # and flip aug_data
    flip_half = tf.image.flip_left_right(aug_data)
    # Concat image and labels
    images_all = tf.concat([original_data, flip_half], 0)
    label = tf.concat([original_label, aug_label], 0)

    return images_all, label
def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    return image, label   

train_data = train_data.map(lambda x, y: img_flip(x, y))

In [ ]:
use_preprocess = False
preprocess_method = clahe_HSV
def preprocess_loader(image, name=None):
    if name == 'None' or name == None:
        return image
    if name == 'clahe_HSV':
        return clahe_HSV(image)
    
    return image
def tf_preprocess(images, preprocess_method):
    images = tf.numpy_function(preprocess_loader, inp=[images, preprocess_method], Tout=[images.dtype])
    return images[0]

def preprocess_images(images, labels, preprocess_method):
    preprocessed_imgs = tf_preprocess(images, preprocess_method)
    return preprocessed_imgs, labels

if use_preprocess:
    train_data = train_data.map(lambda x, y: preprocess_images(x, y, preprocess_method='clahe_HSV'))

# Models

In [ ]:
momentum = 0.9


def fire(x, squeeze, expand):
    y = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
    y = tf.keras.layers.BatchNormalization(momentum=momentum)(y)
    y1 = tf.keras.layers.Conv2D(filters=expand // 2, kernel_size=1, activation='relu', padding='same')(y)
    y1 = tf.keras.layers.BatchNormalization(momentum=momentum)(y1)
    y3 = tf.keras.layers.Conv2D(filters=expand // 2, kernel_size=3, activation='relu', padding='same')(y)
    y3 = tf.keras.layers.BatchNormalization(momentum=momentum)(y3)
    return tf.keras.layers.concatenate([y1, y3])


def fire_module(squeeze, expand):
    return lambda x: fire(x, squeeze, expand)


def build_model(input_img_shape=None, num_classes=None, unfreeze_layers=False, init_from_rand=False, extend_model=False,
                perform_regularization=False, regularizer_type='l1', regularization_penalty=0.001,
                preprocessing_layer=None):
    x = tf.keras.layers.Input(shape=[None, None, 3])  # input is 512x512 pixels RGB
    y = tf.keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation='relu')(x)
    y = tf.keras.layers.BatchNormalization(momentum=momentum)(y)
    y = fire_module(24, 48)(y)
    y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
    y = fire_module(48, 96)(y)
    y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
    y = fire_module(64, 128)(y)
    y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
    y = fire_module(48, 96)(y)
    y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
    y = fire_module(24, 48)(y)
    y = tf.keras.layers.GlobalAveragePooling2D()(y)
    y = tf.keras.layers.Dense(num_classes, activation='sigmoid')(y)
    model = tf.keras.Model(x, y)
    return model

def base_cnn(dim_input, dim_output, perform_regularization=False, regularizer_type='l1',
                  regularization_penalty=0.001, preprocessing_layer=None):

    # Convolution Blocks
    cnn_head = tf.keras.Sequential()

    # Convolutional block 1 (Conv Units - 2)
    cnn_head.add(tf.keras.layers.Conv2D(32, (3, 3), input_shape=dim_input, padding='same', activation='relu'))
    cnn_head.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
    cnn_head.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_head.add(tf.keras.layers.BatchNormalization())


    # Convolutional block 3 (Conv Units - 3)
    cnn_head.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    cnn_head.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    cnn_head.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
    cnn_head.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_head.add(tf.keras.layers.BatchNormalization())

    # Fully Connected Layers
    fc = tf.keras.Sequential()
    fc.add(tf.keras.layers.AveragePooling2D())
    fc.add(tf.keras.layers.Flatten())
    fc.add(tf.keras.layers.Dense(1024, activation='relu'))
    fc.add(tf.keras.layers.Dense(1024, activation='relu'))
    fc.add(tf.keras.layers.Dense(512, activation='relu'))
    fc.add(tf.keras.layers.Dense(dim_output, activation='sigmoid'))

    model = tf.keras.Sequential([
        cnn_head,
        fc
    ])

    # add preprocessing layer
    if preprocessing_layer is not None:
        inputs = tf.keras.Input(shape=dim_input, dtype=tf.uint8)
        x = tf.cast(inputs, tf.float32)
        x = preprocessing_layer(x)
        outputs = model(x)
        model = tf.keras.Model(inputs, outputs)


    return model

# Augmentations


* ***Create a high level order function for the augmentations to be in the model itself such that it would use GPU acceleration***
1. Check why Zoom, sharpen, rotate are not working.
2. Implement cutout, mixup

In [ ]:
from albumentations import VerticalFlip, Flip, MotionBlur, MedianBlur, GlassBlur, GaussNoise, RandomBrightnessContrast, RandomScale, GaussianBlur, RandomBrightness, RandomContrast, ShiftScaleRotate, Compose

# def strong_aug(p=1):
    
#     augs = Compose([Flip(p=p), GaussianBlur(p=p), RandomBrightness(p=p), RandomContrast(p=p), ShiftScaleRotate(p=p)])
#     return augs

# def strong_aug(p=0.5):
#     return Compose([
#         RandomRotate90(),
#         Flip(),
#         Transpose(),
#         OneOf([
#             IAAAdditiveGaussianNoise(),
#             GaussNoise(),
#         ], p=0.2),
#         OneOf([
#             MotionBlur(p=0.2),
#             MedianBlur(blur_limit=3, p=0.1),
#             Blur(blur_limit=3, p=0.1),
#         ], p=0.2),
#         ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
#         OneOf([
#             OpticalDistortion(p=0.3),
#             GridDistortion(p=0.1),
#             IAAPiecewiseAffine(p=0.3),
#         ], p=0.2),
#         OneOf([
# #             CLAHE(clip_limit=2),
#             IAASharpen(),
#             IAAEmboss(),
#             RandomBrightnessContrast(),
#         ], p=0.3),
#         HueSaturationValue(p=0.3),
#     ], p=p)
# transforms = strong_aug(p=0.7)

def augmentations(p=0.5):
    
    transforms = Compose([RandomBrightness(p=p), RandomContrast(p=p), Flip(p=p)])
    return transforms

transforms = augmentations(p=0.5)

In [ ]:
# img = plt.imread(train_filenames[0])
# img_t = augment_image(img)
# # print(img_t.shape)
# print(np.max(img), np.max(img_t))
# plt.imshow(img_t)

In [ ]:
def augment_image(image):
    data = {'image' : image}
    data_transform = transforms(**data)
    img_transformed = data_transform['image']
    return img_transformed

def tf_augment_image(images):
    images = tf.numpy_function(augment_image, inp=[images], Tout=[images.dtype])
    return images[0]

def augment_batches(images, labels):
    # tf.map_fn : takes batches of images, unroll the batches
    # perfom op on every image in batch and rolls back to batch
    aug_images = tf.map_fn(tf_augment_image, images)
    aug_images = tf.reshape(aug_images, shape=tf.shape(images))
    return aug_images, labels

def augment_in_model(images):
    aug_images = tf.map_fn(tf_augment_image, images)
    aug_images = tf.reshape(aug_images, shape=tf.shape(images))
    return aug_images

class AugmentationLayer(tf.keras.layers.Layer):
    
    def __init__(self, augmentation_function, name=None, **kwargs):
        super(AugmentationLayer, self).__init__(name=name)
        self.augmentation_function = augmentation_function
        super(AugmentationLayer, self).__init__(**kwargs)

    def call(self, images, training=False):
        if not training:
            return images
        augmented_images = self.augmentation_function(images)
        return augmented_images
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'augmentation_function': self.augmentation_function 
        })
        return config



# Visualize augmented images

# Callbacks

In [ ]:
def saveModelOnMetric(save_dir, metric='val_loss', mode='min', save_freq='epoch', save_best_only=True,
                      save_weights_only=True):
    checkpoint_path = os.path.join(save_dir, 'model_weights/best_model_on_{}_weights.h5'.format(metric))
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                             monitor=metric,
                                                             save_freq='epoch',
                                                             save_best_only=save_best_only,
                                                             save_weights_only=save_weights_only,
                                                             mode=mode)
    return checkpoint_callback

# learning_rate = tf.keras.experimental.CosineDecay(initial_learning_rate=learning_rate,
#                                                decay_steps=1000)

save_dir = '.'
save_best_model_cbk_on_val_loss = saveModelOnMetric(save_dir,
                                                    metric='val_loss',
                                                    mode='min',
                                                    save_weights_only=False)
save_best_model_cbk_on_val_f1 = saveModelOnMetric(save_dir,
                                                   metric='val_F1',
                                                   mode='max',
                                                 save_weights_only=False)
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(save_dir, 'metrics.csv'), append=True)

######################################################
# write a callback for validation predictions
######################################################
def cosineLearningRateCallback(initial_learning_rate, decay_steps=100000, alpha=0.0, name=None):
    lr_schedule = tf.keras.experimental.CosineDecay(initial_learning_rate=initial_learning_rate,
                                                    decay_steps=decay_steps,
                                                    alpha=alpha,
                                                    name=name)
    return lr_schedule

class metricStatistics(tf.keras.callbacks.Callback):

    def __init__(self, val_data, save_dir, classes, save_freq=10):
        super(metricStatistics, self).__init__()
        self.val_data = val_data
        self.save_dir = save_dir
        self.save_freq = save_freq
        self.classes = classes
        # metrics
        self.f1, self.val_f1 = [], []
        self.loss, self.val_loss = [], []

    # Function for calculating model.predict() on a tf Dataset
    def predict_on_data(self, data, name='train'):
        filenames = []
        if name == 'validation':
            filenames = np.array(list(data.flat_map(
                lambda image, label, filename: tf.data.Dataset.from_tensor_slices(filename)).as_numpy_iterator()))
            data = data.map(lambda image, label, filename: (image, label))
        true_labels = np.array(
            list(data.flat_map(lambda image, label: tf.data.Dataset.from_tensor_slices(label)).as_numpy_iterator()))
        predicted_values = self.model.predict(data)
#         predicted_values = np.reshape(predicted_values, -1)
        predicted_values = self.convert_categorical_to_label(predicted_values)
        if name == 'validation':
            return true_labels, predicted_values, filenames
    

    def save_validation_predictions(self, epoch, labels, values, filenames, name=None):
        if name is None:
            csv_filename = os.path.join(self.save_dir, 'val_predictions_at_epoch_{}.csv'.format(epoch))
        else:
            csv_filename = os.path.join(self.save_dir, name)

        with open(csv_filename, 'a') as fp:
            wr = csv.writer(fp, dialect='excel')
            wr.writerow(['Image name', 'Model Prediction', 'True Label'])
            for i in range(len(labels)):
                try:
                    name = filenames[i].decode('utf-8').split('/')[-1]
                except AttributeError as e:
                    name = filenames[i].split('/')[-1]
                try:
                    wr.writerow([name, round(values[i], 4), labels[i]])
                except:
                    wr.writerow([name, values[i], labels[i]])                    
            fp.close()
    
    
    def unroll_dataloader(self, epoch, logs):
        val_labels, val_values, val_filenames = self.predict_on_data(self.val_data, 'validation')
        return val_labels, val_values, val_filenames
    
    def convert_categorical_to_label(self, predictions):
        labels = []
        predictions[predictions < 0.5] = 0
        predictions[predictions >= 0.5] = 1
        for i in predictions:
            arr = np.nonzero(i)
            lbl = []
            if len(arr[0]) >= 1:
                for j in arr[0]:
                    lbl.append(label_to_cat_label_map[j])
                final_cat_label = ' '.join(lbl)
                labels.append(final_cat_label)
            else:
                labels.append('None')
        return labels

        
    def on_epoch_end(self, epoch, logs=None):
        self.loss.append(logs['loss'])
        self.val_loss.append(logs['val_loss'])
        self.f1.append(logs['F1'])
        self.val_f1.append(logs['val_F1'])

        if epoch != 0 and epoch % self.save_freq == 0:
            val_labels, val_values, val_filenames = self.unroll_dataloader(epoch, logs)
            self.save_validation_predictions(epoch, val_labels, val_values, val_filenames)
            

save_validation_prediction = metricStatistics(save_dir=save_dir, classes=num_classes, save_freq=1, val_data=val_data_with_filenames)

use_learning_rate_decay = True
if use_learning_rate_decay:
    learning_rate = cosineLearningRateCallback(initial_learning_rate=learning_rate)
    
callbacks = [save_best_model_cbk_on_val_f1, 
             save_best_model_cbk_on_val_loss, csv_logger, save_validation_prediction]

# Model

In [ ]:
base_model = tf.keras.applications.MobileNet(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
# base_model.summary()
num_classes = len(unique_labels)
preprocessor = tf.keras.applications.mobilenet.preprocess_input
#add classification head
image_batch, label_batch = next(iter(train_data))
feature_batch = base_model(image_batch)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
prediction_layer = tf.keras.layers.Dense(num_classes, activation='sigmoid')

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])
inputs = tf.keras.Input(shape=(None, None, 3))
# x = data_augmentation(inputs)
# x = AugmentationLayer(augment_in_model)(inputs)
x = preprocessor(inputs)
x = base_model(x)
x = global_average_layer(x)
# x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

# model = base_cnn((512, 512, 3), num_classes)
# model.summary()

In [ ]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)
# loss_fn =  loss=tf.keras.losses.CosineSimilarity(axis=1)
# import focal_loss
# from focal_loss import SparseCategoricalFocalLoss
# loss_fn = SparseCategoricalFocalLoss(gamma=2, from_logits=True)
metrics = ['accuracy', tfa.metrics.F1Score(num_classes=len(unique_labels),
                                           average='macro',
                                          name='F1')]
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# model.compile(optimizer=optimizer, loss=loss_fn,metrics=metrics)
if not os.path.exists('model_weights'):
    os.makedirs('model_weights')
    os.makedirs('metrics')

In [ ]:
base_model.trainable = True
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss=loss_fn,metrics=metrics)
total_epochs = 20
# total_epochs = initial_epochs + finetue_epochs

history_fine = model.fit(train_data.repeat(), 
                         epochs=total_epochs,
                         steps_per_epoch=steps_per_epoch,
#                         initial_epoch=history.epoch[-1],
                         initial_epoch=0,
                        validation_data=val_data,
                         callbacks=callbacks,)
#                        class_weight=class_weights,)
#                         batch_size=8)

'''
1. No AUG400s
2. with AUG 478 seconds
3. With data split - 420
'''

In [ ]:
model.save('model_weights/final_model.h5')
acc = history_fine.history['accuracy']
val_acc = history_fine.history['val_accuracy']
loss = history_fine.history['loss']
val_loss = history_fine.history['val_loss']
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
# plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
# plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
test_path= ('../input/plant-pathology-2021-fgvc8/test_images/')
test_filenames = os.listdir(test_path)
test_abs_filenames = [os.path.join(test_path, name) for name in test_filenames]
image_size = img_size
def test_get_image_dontnormalize(filename, image_size=256, preprocess_method=None):
    # decode_jpeg
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string)  # decodes to uint8 tensor

#     preprocessed = tf_preprocess(image_decoded, preprocess_method)
#     preprocessed = tf.reshape(preprocessed, tf.shape(image_decoded))

    # resize image
    image_resized = tf.cast(tf.image.resize(image_decoded, (image_size, image_size)),
                            tf.uint8)  # model expects int [0,255]

    return image_resized

AUTOTUNE = tf.data.experimental.AUTOTUNE
img_size_tiled = tf.constant([image_size] * len(test_filenames))
test_ds = tf.data.Dataset.from_tensor_slices((tf.constant(test_abs_filenames), img_size_tiled)).map(test_get_image_dontnormalize, num_parallel_calls=AUTOTUNE)
test_data = test_ds.batch(64)

In [ ]:
predictions = model.predict(test_data)